## Memory Module Construction

Essentially a searchable summarizer of all conversations.

In [9]:
from training_utils import load_and_sort_conversations
from IMPersona.memory_module import MemoryModule
import json

In [10]:
directory = 'data/imessage/parsed_combined'
recent_convos = load_and_sort_conversations(
    directory, 
    num_conversations=6000, 
    min_messages=5,
    max_messages=300,
)
recent_convos = list(reversed(recent_convos))

Loaded 5587 conversations with 5-300 messages
Kept 5587 most recent conversations


In [11]:
import tiktoken

# Initialize the tokenizer (using gpt-3.5-turbo's tokenizer as an example)
encoding = tiktoken.get_encoding("cl100k_base")  # This is the encoding used by gpt-3.5-turbo and gpt-4

total_tokens = 0
for conversation in recent_convos:
    for message in conversation:
        # Format each message as "timestamp speaker: content"
        formatted_message = f"{message['timestamp']} {message['speaker']}: {message['content']}"
        # Count tokens in the formatted message
        num_tokens = len(encoding.encode(formatted_message))
        total_tokens += num_tokens

print(f"Total number of tokens: {total_tokens}")

for idx, conversation in enumerate(recent_convos):
    conversation_tokens = 0
    for message in conversation:
        formatted_message = f"{message['timestamp']} {message['speaker']}: {message['content']}"
        num_tokens = len(encoding.encode(formatted_message))
        conversation_tokens += num_tokens
    
    if conversation_tokens > 100000:
        print(f"Warning: Conversation {idx} exceeds token limit with {conversation_tokens} tokens")

Total number of tokens: 3868564


In [30]:
with open('IMPersona/data/ben/conversation_store_full_5-300.json', 'w') as f:
    json.dump(recent_convos, f)

#### Processing Conversations ####

In [1]:
from IMPersona.memory_module import MemoryModule
import json
import nest_asyncio 
nest_asyncio.apply()

async def process_conversation_batch(memory, conversations, batch_size=100):
    """Process conversations in batches to avoid overwhelming the API"""
    for i in range(0, len(conversations), batch_size):
        batch = conversations[i:i + batch_size]
        print(f"\n--- Processing Batch {i//batch_size + 1}, Conversations {i} to {i + len(batch)} ---")
        await memory.async_process_conversations(batch)
        print(f"\nCurrent memory counts:")
        print(f"Memories: {len(memory.memories)}")

In [ ]:
memory = MemoryModule(
        conversation_store_file_path='IMPersona/data/ben/conversation_store_full_5-300.json',
        impersonation_name="Ben",
        model_name="gpt-4o",
        memory_file_path="IMPersona/data/ben/memory_gpt_4o.json",
    )

start_idx = 0
end_idx = None # not inclusive
batch_size = 100

# Run the async processing
try:
    # Load conversations from the store
    with open('IMPersona/data/ben/conversation_store_full_5-300.json', 'r') as f:
        conversations = json.load(f)
    
    # Validate and adjust end_idx if needed
    if end_idx is None:
        end_idx = len(conversations)
    end_idx = min(end_idx, len(conversations))
        
    print(f"\nFound {len(conversations)} total conversations")
    print(f"Processing conversations from index {start_idx} to {end_idx}")
    
    # Process the conversations asynchronously
    selected_conversations = conversations[start_idx:end_idx]
    await process_conversation_batch(memory, selected_conversations, batch_size=batch_size)  # Using await instead of asyncio.run()

except Exception as e:
    print(f"Error: An unexpected error occurred: {str(e)}")

### Consolidate Memories: By Day

In [1]:
from IMPersona.memory_module import MemoryModule

# Initialize the memory module with your existing memory file
memory_module = MemoryModule(
    conversation_store_file_path="IMPersona/data/ben/conversation_store_full_5-300.json",
    impersonation_name="Ben",  # or whatever name you're using
    model_name="gpt-4o",  # or whatever model you're using
    memory_file_path="IMPersona/data/ben/memory_gpt_4o.json"  # your current memory file
)

In [2]:
# Consolidate the memories by day
await memory_module.async_consolidate_by_day_batched(batch_size=100)

Starting batched memory consolidation...
Processing 62019 total memories...
Grouped memories into 12385 distinct dates


Consolidating memories by date:   1%|          | 100/12385 [00:28<14:46:05,  4.33s/it]


Saving intermediate progress (139 memories so far)...


Consolidating memories by date:   2%|▏         | 300/12385 [00:51<1:16:43,  2.63it/s] 


Saving intermediate progress (397 memories so far)...


...


Saving intermediate progress (17844 memories so far)...


Consolidating memories by date:  99%|█████████▉| 12300/12385 [27:52<00:56,  1.50it/s]


Saving intermediate progress (18131 memories so far)...


Consolidating memories by date: 100%|██████████| 12385/12385 [28:07<00:00,  7.34it/s]


Consolidation complete. Reduced 62019 memories to 18245
Consolidated memories saved to: IMPersona/data/ben/memory_gpt_4o_consolidated.json


In [3]:
from IMPersona.memory_module import MemoryModule
import json
# Initialize the memory module with your existing memory file
memory_module = MemoryModule(
    conversation_store_file_path="IMPersona/data/ben/conversation_store_full_5-300.json",
    impersonation_name="Ben",  # or whatever name you're using
    model_name="gpt-4o",  # or whatever model you're using
    memory_file_path="IMPersona/data/ben/memory_gpt_4o_consolidated.json"  # your current memory file
)

In [4]:
memory_by_day = memory_module.group_memories_by_day()

with open('IMPersona/data/ben/memory_by_day.json', 'w') as f:
    json.dump(memory_by_day, f)